In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num , DateFormatter
import matplotlib.dates as mdates
from sklearn.preprocessing import StandardScaler
import tensorflow as tf


from src.globals import processed_data_dir
from src.visualization import plot_metrics

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

In [4]:
# check GPU visibility
tf.config.get_visible_devices(
    device_type='GPU'
)

[]

In [5]:
train_dir = processed_data_dir / 'train'
val_dir = processed_data_dir / 'val'
test_dir = processed_data_dir / 'test'

In [21]:
from src.helpers import WindowGenerator

In [22]:
w1 = WindowGenerator(input_width=24, label_width=6, shift=6,
                     train_dir=train_dir, val_dir=val_dir, test_dir=test_dir,
                     label_columns=['Lufttemperatur [GradC]'],
                     embedding_column='Location')
w1

{'Luftdruck [hPa]': 0, 'Lufttemperatur [GradC]': 1, 'rel. Luftfeuchte [%]': 2, 'Wx': 3, 'Wy': 4, 'max Wx': 5, 'max Wy': 6, 'Day sin': 7, 'Day cos': 8, 'Year sin': 9, 'Year cos': 10, 'Location': 11}
['Luftdruck [hPa]', 'Lufttemperatur [GradC]', 'rel. Luftfeuchte [%]', 'Wx', 'Wy', 'max Wx', 'max Wy', 'Day sin', 'Day cos', 'Year sin', 'Year cos', 'Location']
11 :  Location


Total window size: 30
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29]
Label column name(s): ['Lufttemperatur [GradC]']

In [23]:
ds = w1.csv_reader(train_dir)

for i in ds.take(1):
    print(i)

((<tf.Tensor: shape=(24, 11), dtype=float32, numpy=
array([[ 1.9965459e-01, -6.2726003e-01,  9.7017813e-01, -3.3146796e-01,
        -1.8577446e-01, -3.0065876e-01, -1.8367843e-01, -2.0892539e-05,
         1.4143265e+00, -4.0802453e-03,  1.4158330e+00],
       [ 1.9965459e-01, -6.7416644e-01,  1.0835139e+00, -4.0234268e-02,
        -3.3869189e-01, -2.4140520e-02, -2.7899554e-01,  3.6599547e-01,
         1.3661371e+00, -3.0669540e-03,  1.4158354e+00],
       [ 1.9965459e-01, -6.8589300e-01,  1.1401818e+00, -3.6569142e-01,
         3.7935230e-01, -4.6147925e-01,  4.5405185e-01,  7.0706838e-01,
         1.2248532e+00, -2.0536615e-03,  1.4158370e+00],
       [ 1.9965459e-01, -7.0934623e-01,  1.1401818e+00, -2.2151475e-04,
        -2.1305431e-01,  6.5344229e-02, -4.5728460e-01,  9.9995440e-01,
         1.0001026e+00, -1.0403680e-03,  1.4158379e+00],
       [ 1.9965459e-01, -8.6179203e-01,  1.1401818e+00,  1.9974042e-01,
        -6.1268353e-01,  2.2872269e-01, -6.4047241e-01,  1.2246937e+00,


In [ ]:
### template from thesis code - not yet working

In [25]:
tf.keras.backend.clear_session()

# Node Embeddings
embedding_inputs = tf.keras.Input(shape =(24,))
embedding_layer = tf.keras.layers.Embedding(input_dim=max(train_df.Location) + 1, 
                                            output_dim=2, input_length=24)(embedding_inputs)

# Encoder Inputs
feature_inputs = tf.keras.layers.Input(shape=(24, 11))

# Concat Embeddings & Encoder Inputs
concat_layer = tf.keras.layers.Concatenate()([feature_inputs, embedding_layer])

layer1 = tf.keras.layers.LSTM(30, return_sequences=True)(concat_layer)

output = tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(units=6, activation='linear'))

output = output(layer1)

model = tf.keras.Model(inputs=[feature_inputs, embedding_inputs], outputs=output)

NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported